In [1]:
import numpy as np
import pandas

data_train = pandas.read_csv('salary-train.scv')
data_test = pandas.read_csv('salary-test-mini.csv')

data_train['FullDescription'] = data_train['FullDescription'].str.lower()
data_test['FullDescription'] = data_test['FullDescription'].str.lower()

data_train['FullDescription'] = data_train['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)
data_test['FullDescription'] = data_test['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)

data_train['LocationNormalized'].fillna('nan', inplace=True)
data_train['ContractTime'].fillna('nan', inplace=True)
data_train.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,international sales manager london k ...,London,permanent,33000
1,an ideal opportunity for an individual that ha...,London,permanent,50000
2,online content and brand manager luxury reta...,South East London,permanent,40000
3,a great local marketleader is seeking a perman...,Dereham,permanent,22500
4,registered nurse rgn nursing home for young...,Sutton Coldfield,nan,20355


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=5)
X_text = vectorizer.fit_transform(data_train['FullDescription'].values)
X_text_test = vectorizer.transform(data_test['FullDescription'].values)

In [15]:
from sklearn.feature_extraction import DictVectorizer

# Size of 'LocationNormalized' + 'ContractTime' after one-hot encoding
print len(data_train[['LocationNormalized']].stack().value_counts().index.tolist())
print len(data_train[['ContractTime']].stack().value_counts().index.tolist())

dict_vectorizer = DictVectorizer(sparse=False)
X_train_categ = dict_vectorizer.fit_transform(data_train[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test_categ = dict_vectorizer.transform(data_test[['LocationNormalized', 'ContractTime']].to_dict('records'))
len(X_train_categ[0])

1763
3


1766

In [18]:
from scipy.sparse import hstack

X_train = hstack([X_text, X_train_categ])
X_test = hstack([X_text_test, X_test_categ])
X_train.shape

(60000, 24627)

In [19]:
from sklearn.linear_model import Ridge

clf = Ridge(alpha=1, random_state=241)
clf.fit(X_train, data_train['SalaryNormalized'].values)

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=241, solver='auto', tol=0.001)

In [20]:
data_test

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,we currently have a vacancy for an hr project ...,Milton Keynes,contract,NaN
1,a web developer opportunity has arisen with an...,Manchester,permanent,NaN


In [22]:
np.round(clf.predict(X_test), 2)

array([ 56555.62,  37188.32])